In [1]:
import pandas as pd
import re
import os
from datetime import datetime

def preprocess_ingredient_data(df, ingredients_column='ingredients', disease_column='disease'):
    """
    預處理步驟：如果 ingredients 欄位為空，嘗試從 disease 欄位複製草藥資料
    
    參數:
    df: 原始 DataFrame
    ingredients_column: 草藥成分欄位名稱
    disease_column: 疾病欄位名稱（可能包含誤分類的草藥資料）
    
    返回:
    處理後的 DataFrame 和處理統計資訊
    """
    print("=== 執行資料預處理 ===")
    
    # 複製 DataFrame 避免修改原始資料
    processed_df = df.copy()
    
    # 統計資訊
    total_rows = len(df)
    empty_ingredients_count = 0
    transferred_count = 0
    
    # 識別草藥格式的正規表達式模式
    # 如果 disease 欄位包含〈〉或<>符號，很可能是草藥資料
    herb_indicators = [
        r'[〈<].*[〉>]',  # 包含劑量標記
        r'[\u4e00-\u9fa5]{2,}\s+[\u4e00-\u9fa5]{2,}',  # 多個中文草藥名稱
        r'(各|共|君|臣|佐|使)',  # 中醫術語
    ]
    
    for index, row in processed_df.iterrows():
        ingredients_value = row[ingredients_column] if ingredients_column in row else None
        disease_value = row[disease_column] if disease_column in row else None
        
        # 檢查 ingredients 是否為空
        is_ingredients_empty = (
            pd.isna(ingredients_value) or 
            str(ingredients_value).strip() == '' or 
            str(ingredients_value) == 'nan'
        )
        
        if is_ingredients_empty:
            empty_ingredients_count += 1
            
            # 檢查 disease 欄位是否包含疑似草藥資料
            if not pd.isna(disease_value) and str(disease_value).strip():
                disease_text = str(disease_value).strip()
                
                # 檢查是否符合草藥格式特徵
                looks_like_herbs = any(re.search(pattern, disease_text) for pattern in herb_indicators)
                
                if looks_like_herbs:
                    # 將 disease 的內容複製到 ingredients
                    processed_df.at[index, ingredients_column] = disease_text
                    # 清空 disease 欄位（或者保持原值，看需求）
                    # processed_df.at[index, disease_column] = ''
                    transferred_count += 1
                    print(f"  第 {index+1} 行：從 disease 轉移草藥資料到 ingredients")
                    print(f"    轉移內容: {disease_text[:50]}{'...' if len(disease_text) > 50 else ''}")
    
    print(f"\n預處理統計:")
    print(f"  總行數: {total_rows}")
    print(f"  ingredients 為空的行數: {empty_ingredients_count}")
    print(f"  成功轉移的行數: {transferred_count}")
    
    return processed_df, {
        'total_rows': total_rows,
        'empty_ingredients': empty_ingredients_count,
        'transferred': transferred_count
    }
def split_ingredients_to_rows(df, ingredients_column='ingredients'):
    """
    將包含多個草藥的 ingredients 欄位拆分成多行
    
    參數:
    df: 原始 DataFrame（已經過預處理）
    ingredients_column: 包含草藥資料的欄位名稱
    
    返回:
    拆分後的 DataFrame，新增 '草藥' 和 '劑量說明' 欄位
    """
    
    # 準備儲存結果的列表
    expanded_rows = []
    
    # 遍歷每一行
    for index, row in df.iterrows():
        ingredients_text = str(row[ingredients_column])
        
        # 如果該欄位為空或無效，保留原行但新增空的草藥欄位
        if pd.isna(row[ingredients_column]) or ingredients_text.strip() == '' or ingredients_text == 'nan':
            new_row = row.copy()
            new_row['草藥'] = ''
            new_row['劑量說明'] = ''
            expanded_rows.append(new_row)
            continue
        
        # 使用正規表達式解析草藥和劑量
        # 匹配模式：草藥名稱 + 空格(可選) + 〈劑量〉或<劑量>
        # 也處理沒有劑量的情況
        # 支援全形〈〉和半形<>兩種格式
        herb_pattern = r'([^〈〉<>\s]+)\s*(?:〈([^〈〉]*)〉|<([^<>]*)>)?'
        
        # 先進行基本的清理，移除多餘空格
        cleaned_text = re.sub(r'\s+', ' ', ingredients_text.strip())
        
        # 找出所有匹配的草藥和劑量
        matches = re.findall(herb_pattern, cleaned_text)
        
        if matches:
            # 為每個草藥創建一行
            for match in matches:
                herb_name = match[0].strip()  # 草藥名稱
                # 劑量可能在 match[1] (〈〉格式) 或 match[2] (<>格式)
                dosage = (match[1] or match[2] or '').strip()
                
                # 跳過空的草藥名稱
                if not herb_name:
                    continue
                    
                # 複製原行的所有資料
                new_row = row.copy()
                new_row['草藥'] = herb_name
                new_row['劑量說明'] = dosage
                expanded_rows.append(new_row)
        else:
            # 如果無法解析，保留原行
            new_row = row.copy()
            new_row['草藥'] = ingredients_text
            new_row['劑量說明'] = ''
            expanded_rows.append(new_row)
    
    # 轉換為 DataFrame
    result_df = pd.DataFrame(expanded_rows)
    
    # 重新排序欄位，將新增的欄位放在前面
    cols = result_df.columns.tolist()
    if '草藥' in cols and '劑量說明' in cols:
        cols.remove('草藥')
        cols.remove('劑量說明')
        new_cols = ['草藥', '劑量說明'] + cols
        result_df = result_df[new_cols]
    
    return result_df

def process_files_with_ingredient_splitting():
    """
    主處理函數：掃描並處理包含 ingredients 欄位的檔案
    """
    
    # --- 使用者設定區 ---
    FOLDER_PATH = '.'  # 當前資料夾
    FILE_PATTERN = 'classified_section_卷'  # 檔案名稱模式
    INGREDIENTS_COLUMN = 'ingredients'  # 要拆分的欄位名稱
    
    print("=== 草藥成分拆分器啟動 ===")
    print(f"掃描資料夾: {os.path.abspath(FOLDER_PATH)}")
    print(f"尋找檔案模式: {FILE_PATTERN}*.xlsx")
    
    # 掃描目標檔案
    target_files = []
    for filename in os.listdir(FOLDER_PATH):
        if filename.startswith(FILE_PATTERN) and filename.endswith('.xlsx'):
            target_files.append(filename)
    
    if not target_files:
        print(f"錯誤：在資料夾中找不到符合 '{FILE_PATTERN}*.xlsx' 格式的檔案。")
        return
    
    target_files.sort()  # 按檔名排序
    print(f"找到 {len(target_files)} 個目標檔案:")
    for file in target_files:
        print(f"  - {file}")
    
    # 處理每個檔案
    processed_count = 0
    for filename in target_files:
        file_path = os.path.join(FOLDER_PATH, filename)
        print(f"\n--- 正在處理: {filename} ---")
        
        try:
            # 讀取檔案
            df = pd.read_excel(file_path)
            print(f"原始資料: {len(df)} 行, {len(df.columns)} 欄")
            
            # 檢查是否有 ingredients 和 disease 欄位
            required_columns = [INGREDIENTS_COLUMN]
            missing_columns = [col for col in required_columns if col not in df.columns]
            
            if missing_columns:
                print(f"  警告: 檔案中找不到必要欄位 {missing_columns}，跳過此檔案。")
                print(f"  可用欄位: {list(df.columns)}")
                continue
            
            # 步驟1: 預處理 - 檢查是否需要從 disease 轉移資料
            if 'disease' in df.columns:
                print("  執行預處理：檢查 disease 欄位中的草藥資料...")
                df, preprocess_stats = preprocess_ingredient_data(df, INGREDIENTS_COLUMN, 'disease')
                
                if preprocess_stats['transferred'] > 0:
                    print(f"  ✓ 成功從 disease 欄位轉移 {preprocess_stats['transferred']} 筆草藥資料")
                else:
                    print("  → 沒有發現需要轉移的草藥資料")
            else:
                print("  未發現 disease 欄位，跳過預處理步驟")
            
            # 顯示拆分前的範例
            print("  拆分前範例 (前5行的 ingredients 內容):")
            for i, ingredient in enumerate(df[INGREDIENTS_COLUMN].head().fillna('')):
                print(f"    {i+1}. {ingredient}")
            
            # 步驟2: 執行拆分
            print("  開始執行草藥拆分...")
            expanded_df = split_ingredients_to_rows(df, INGREDIENTS_COLUMN)
            print(f"拆分後資料: {len(expanded_df)} 行, {len(expanded_df.columns)} 欄")
            
            # 顯示拆分後的範例
            print("  拆分後範例 (前10行):")
            display_cols = ['草藥', '劑量說明']
            if len(expanded_df.columns) > 2:
                display_cols.append(expanded_df.columns[2])  # 加入第一個原始欄位作為參考
            
            for i, row in expanded_df[display_cols].head(10).iterrows():
                print(f"    {i+1}. 草藥: '{row['草藥']}' | 劑量: '{row['劑量說明']}' | 參考: '{row[display_cols[2]] if len(display_cols) > 2 else 'N/A'}'")
            
            # 生成輸出檔名
            base_name = filename.replace('.xlsx', '')
            today_str = datetime.now().strftime("%Y%m%d_%H%M")
            output_filename = f"{base_name}_草藥拆分_{today_str}.xlsx"
            output_path = os.path.join(FOLDER_PATH, output_filename)
            
            # 儲存結果
            expanded_df.to_excel(output_path, index=False)
            print(f"  ✓ 成功儲存至: {output_filename}")
            processed_count += 1
            
        except Exception as e:
            print(f"  ✗ 處理檔案 '{filename}' 時發生錯誤: {e}")
            continue
    
    print(f"\n=== 處理完成 ===")
    print(f"成功處理 {processed_count} 個檔案")

def demo_ingredient_parsing():
    """
    示範草藥解析功能，並輸出示範檔案
    """
    print("\n=== 草藥解析示範 ===")
    
    # 模擬測試資料 - 包含您提到的實際格式
    test_data = [
        "草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香",
        "當歸<10g> 川芎<6g> 白芍<12g>",
        "人參 黃耆〈15g〉 甘草",
        "桂枝〈3g〉白朮〈9g〉茯苓〈12g〉炙甘草〈6g〉",
        "附子 熟地黃〈15g〉 山茱萸〈10g〉",
        "",
        "單味草藥",
        "大黃〈6g〉芒硝〈9g〉枳實〈12g〉厚朴〈24g〉",
        "麻黃 桂枝〈6g〉 杏仁〈9g〉 炙甘草〈3g〉"
    ]
    
    # 創建測試 DataFrame
    test_df = pd.DataFrame({
        'id': range(1, len(test_data) + 1),
        'formula_name': [f'方劑{i}' for i in range(1, len(test_data) + 1)],
        'category': ['補血劑', '補氣劑', '溫裡劑', '補腎劑', '空白測試', '單方', '攻下劑', '解表劑'],
        'ingredients': test_data,
        'usage': ['飯後服', '空腹服', '溫服', '鹽水送服', '', '研末服', '大便通為度', '微汗為度']
    })
    
    print("原始測試資料:")
    print(test_df)
    
    # 執行拆分
    result_df = split_ingredients_to_rows(test_df)
    
    print(f"\n拆分結果 (從 {len(test_df)} 行拆分成 {len(result_df)} 行):")
    print(result_df[['草藥', '劑量說明', 'id', 'formula_name', 'category']])
    
    # 生成示範檔案
    today_str = datetime.now().strftime("%Y%m%d_%H%M")
    
    # 儲存原始資料
    original_filename = f"草藥拆分示範_原始資料_{today_str}.xlsx"
    test_df.to_excel(original_filename, index=False)
    print(f"\n✓ 原始示範資料已儲存至: {original_filename}")
    
    # 儲存拆分結果
    result_filename = f"草藥拆分示範_拆分結果_{today_str}.xlsx"
    result_df.to_excel(result_filename, index=False)
    print(f"✓ 拆分結果已儲存至: {result_filename}")
    
    # 顯示統計資訊
    print(f"\n=== 拆分統計 ===")
    print(f"原始資料行數: {len(test_df)}")
    print(f"拆分後行數: {len(result_df)}")
    print(f"總共識別出草藥數量: {len(result_df[result_df['草藥'] != ''])}")
    print(f"有劑量資訊的草藥數量: {len(result_df[result_df['劑量說明'] != ''])}")
    
    return original_filename, result_filename

if __name__ == "__main__":
    # 提供兩個選項
    print("請選擇執行模式:")
    print("1. 處理實際檔案")
    print("2. 示範解析功能")
    
    choice = input("請輸入選項 (1 或 2): ").strip()
    
    if choice == "1":
        process_files_with_ingredient_splitting()
    elif choice == "2":
        demo_ingredient_parsing()
    else:
        print("執行示範模式...")
        demo_ingredient_parsing()

請選擇執行模式:
1. 處理實際檔案
2. 示範解析功能
=== 草藥成分拆分器啟動 ===
掃描資料夾: /home/nculcwu/DeepSeek
尋找檔案模式: classified_section_卷*.xlsx
找到 1 個目標檔案:
  - classified_section_卷168.xlsx

--- 正在處理: classified_section_卷168.xlsx ---
原始資料: 50 行, 8 欄
  執行預處理：檢查 disease 欄位中的草藥資料...
=== 執行資料預處理 ===
  第 3 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 療百病虎骨　野葛〈各三兩〉附子〈一十五枚重九斤〉甘草　細辛〈各一兩〉杏仁　巴豆〈去皮心〉芎藭〈切各一...
  第 12 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治五種氣積及五噎胸膈不快傍痰宿飲丁香　木香　沉香　檳榔　官桂　胡椒　硼砂〈研〉白丁香〈各一分〉肉豆䓻...
  第 29 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治積聚結實腹滿刺痛泄痢不止〈方見諸氣門諸氣類〉
  第 32 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治脇下邪氣積聚寒熱往來如温瘧怛山　蜀漆　白薇　桂心　鮀甲　附子　白术　鱉甲　鹿蟲　貝齒〈各一兩〉蜚蝱...
  第 38 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治一切積滯痰逆惡心霍亂吐瀉膈氣痞滿脇肋積塊胸膈膨悶嘔噦心痰泄痢不止代赭石〈煅醋碎研〉礞石〈研各一兩〉...
  第 39 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治積順氣寛中消積滯化痰飲治中腕氣痞心腹堅脹脇下𦂳硬胸中痞塞喘滿短氣喑心氣不通嘔吐痰逆飲食不下大便不調...
  第 46 行：從 disease 轉移草藥資料到 ingredients
    轉移內容: 治一切積滯痰逆惡心吐㵼霍亂膈氣痞滿脇肋積塊胸膈膨悶嘔噦心疼泄㵼下痢大㦸　龍膽　木香〈各半兩〉杏仁〈去...

預處理統計:
  總行數: 50
  ingredients 為空的行數: 9
  成功轉